In [1]:
import pandas as pd

import pulp
import ast


import pandas as pd
import numpy as np


from requests.models import PreparedRequest
from sklearn.cluster import KMeans
from kneed import KneeLocator 

from geopy.distance import geodesic
from kneed import KneeLocator 
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

import networkx as nx
import pandas as pd
import numpy as np

import requests
import pulp
import folium
import ast
import heapq

In [2]:
source = pd.read_csv("/Users/samuele/Desktop/我/CC/FoliaStream/test_ahc/source.csv")
sink = pd.read_csv("/Users/samuele/Desktop/我/CC/FoliaStream/test_ahc/sink.csv")
matrix = pd.read_csv("/Users/samuele/Desktop/我/CC/FoliaStream/test_ahc/matrix.csv").drop('Unnamed: 0.1', axis=1)

In [4]:
quantity_cost_segments={
  'pipe':{
    (0, 50000): 2.5,
    (50000, 100000): 1.5,
    (100000, 250000): 0.9,
    (250000, 500000): 0.6,
    (500000, 1000000): 0.5,
    (1000000, 2000000): 0.3,
    (2000000, 999999999): 0.2},
  'truck_ship':{
    (0, 50000): 0.3,
    (50000, 100000): 0.3,
    (100000, 250000): 0.3,
    (250000, 500000): 0.3,
    (500000, 1000000): 0.3,
    (1000000, 2000000): 0.3,
    (2000000, 999999999): 0.3}
}

In [5]:

def network_optimization_levelized(df_source, df_sink, df_cost_matrix, source_id, sink_id, source_capacity, sink_capacity, emission_cost, transport_method, quantity_cost_segments, capture_cost, stock_cost):

    df_source[source_id] = df_source[source_id].astype(int)
    df_sink[sink_id] = df_sink[sink_id].astype(int)

    df_cost_matrix = df_cost_matrix.rename(columns={"Unnamed: 0":sink_id})
    
    # Network initialization
    network = pulp.LpProblem("Network_problem", pulp.LpMinimize)

    # Generate nodes
    source_list = df_source[source_id].astype(str)
    sink_list = list(df_sink[sink_id].astype(str))
    sink_list.append("Atmosphere")  

    nodes = []
    for i in source_list:
        nodes.append(i)
    for i in sink_list:
        nodes.append(i)
    
    # Generate arcs
    arcs = []
    arc_capacities = {}

    for i in source_list:
        for j in sink_list:
            if j != "Atmosphere":
                arcs.append((f"source_id_{i}", f"sink_id_{j}"))
                arc_capacities[(f"source_id_{i}", f"sink_id_{j}")] = 10000000 # 10Mty capacity of pipes
        arcs.append((f"source_id_{i}","Atmosphere"))
        arc_capacities[(f"source_id_{i}","Atmosphere")] = 99999999999999999999999 # infinite value for emitting

    
    # Flow variables
    flow_vars = pulp.LpVariable.dicts("Flow", arcs, 0, None, pulp.LpContinuous)

    
    # Cost functions
    for method, data in quantity_cost_segments.items():
        if type(list(quantity_cost_segments[method].keys())[0]) == str:
            quantity_cost_segments[method] = {ast.literal_eval(k): v for k, v in quantity_cost_segments[method].items()}
        else:
            pass

    # Set sink_id as index of transport cost matrix
    transport_cost = df_cost_matrix.set_index(sink_id)

    cost_segments = {}
    for i in source_list:
        for j in sink_list:

            cost_segments[(f"source_id_{i}",f"sink_id_{j}")] = [
                (0,50000,quantity_cost_segments[transport_method][(0, 50000)]*transport_cost.at[j,i]),
                (50000,100000,quantity_cost_segments[transport_method][(50000,100000)]*transport_cost.at[j,i]),
                (100000,250000,quantity_cost_segments[transport_method][(100000,250000)]*transport_cost.at[j,i]),
                (250000,500000,quantity_cost_segments[transport_method][(250000,500000)]*transport_cost.at[j,i]),
                (500000,1000000,quantity_cost_segments[transport_method][(500000,1000000)]*transport_cost.at[j,i]),
                (1000000,2000000,quantity_cost_segments[transport_method][(1000000,2000000)]*transport_cost.at[j,i]),
                (2000000,999999999,quantity_cost_segments[transport_method][(2000000,999999999)]*transport_cost.at[j,i]),
            ]
        cost_segments[(f"source_id_{i}",f"Atmosphere")] = [(0,1000000000000, 0)]
    

    # Segment variables
    segment_vars = {}
    for arc in arcs:
        segment_vars[arc] = []
        for i, (start, end, slope) in enumerate(cost_segments[arc]):
            var = pulp.LpVariable(f"Segment_{arc}_{i}", 0, end - start, pulp.LpContinuous)
            segment_vars[arc].append((var, start, end, slope))
        
    # Objective function
    # network += pulp.lpSum([var * slope for arc in arcs for var, start, end, slope in segment_vars[arc]]), "TotalCost"
    
    network += (
        pulp.lpSum(flow_vars[arc] * capture_cost for arc in arcs if arc[1] != "Atmosphere") +  # Capture
        pulp.lpSum(var * slope for arc in arcs for (var, _, _, slope) in segment_vars[arc]) +  # Transport
        pulp.lpSum(flow_vars[arc] * stock_cost for arc in arcs if arc[1] != "Atmosphere") +  # Capture
        pulp.lpSum(flow_vars[arc] * emission_cost for arc in arcs if arc[1] == "Atmosphere")    # Emission
    ), "TotalCost"


    # Constraints
    for i, rsource in df_source.iterrows():
        network += sum(flow_vars[(f"source_id_{int(rsource[source_id])}", f"sink_id_{int(rsink[sink_id])}")] for j, rsink in df_sink.iterrows() if int(rsink[sink_capacity]) != "Atmosphere") + flow_vars[(f"source_id_{int(rsource[source_id])}", "Atmosphere")] == rsource[source_capacity], f"source_id_{int(rsource[source_id])}_outflow"
    
    for i, rsink in df_sink.iterrows():
            network += sum(flow_vars[(f"source_id_{int(rsource[source_id])}", f"sink_id_{int(rsink[sink_id])}")] for j, rsource in df_source.iterrows()) <= int(rsink[sink_capacity]), f"sink_id_{int(rsink[sink_id])}_inflow"

    for arc in arcs:
        network += flow_vars[arc] <= arc_capacities[arc], f"Capacity_{arc}"

    for arc in arcs:
        network += flow_vars[arc] == pulp.lpSum([var for var, start, end, slope in segment_vars[arc]]), f"Piecewise_{arc}"
    
    # Solution
    network.solve()

    # Check status
    status = pulp.LpStatus[network.status]
    print(status)

    # if status == "Infeasible":
    #     print("Infeasible model")
    # else:
    #     for arc in arcs:
    #         print(f"Flow on arc {arc}: {flow_vars[arc].varValue}")
    #     print("Total Cost: ", pulp.value(network.objective))

    # Export results
    results = []

    for arc in arcs:
        if flow_vars[arc].varValue > 0:
            if arc[1] != "Atmosphere":
                results.append({
                    'source_id':arc[0][10:],
                    'sink_id':arc[1][8:],
                    'co2_transported':flow_vars[arc].varValue
                })
            else:
                results.append({
                    'source_id':arc[0][10:],
                    'sink_id':arc[1],
                    'co2_transported':flow_vars[arc].varValue
                })
            
    df_results = pd.DataFrame(results)

    return df_results

In [6]:
emission_cost = 100
capture_cost = 100
stock_cost = 15

In [7]:
network_optimization_levelized(source, 
                               sink, 
                               matrix, 
                               'id', 
                               'id', 
                               'emission', 
                               'sum_mid', 
                               emission_cost, 
                               'pipe', 
                               quantity_cost_segments, 
                               capture_cost, 
                               stock_cost)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/2d/n_vsfm_n1_l_wtnsx7bt2g300000gn/T/01485c42a7a0450585d102a02fd34c1c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/2d/n_vsfm_n1_l_wtnsx7bt2g300000gn/T/01485c42a7a0450585d102a02fd34c1c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5196 COLUMNS
At line 53014 RHS
At line 58206 BOUNDS
At line 75796 ENDATA
Problem MODEL has 5191 rows, 20130 columns and 27720 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2113 (-3078) rows, 16703 (-3427) columns and 18815 (-8905) elements
0  Obj 0 Primal inf 18908984 (33)
33  Obj 1.8908984e+09
Optimal - objective value 1.8908984e+09
After Postsolve, objective 1.8908984e+09, infeasibilities - dual 0 (0), primal 0 (0)
Optimal obj

,source_id,sink_id,co2_transported
0,1897479,Atmosphere,2794440.400
1,1897478,Atmosphere,1036067.600
2,32438922,Atmosphere,960096.780
3,1897504,Atmosphere,883825.440
4,32438914,Atmosphere,802542.670
5,1897481,Atmosphere,802405.950
6,1897492,Atmosphere,784826.070
7,32438910,Atmosphere,784710.700
8,1897508,Atmosphere,757999.270
9,32438894,Atmosphere,720891.750


In [113]:
url = 'http://router.project-osrm.org/table/v1/driving/'
transport_cost = {'pipe': {'less_180': 0.08, 'range_180_500': 0.12, 'range_500_750': 0.15, 'range_750_1500': 0.18, 'more_1500': 0.24}, 'truck_ship': {'less_180': 0.36, 'range_180_500': 0.36, 'range_500_750': 0.36, 'range_750_1500': 0.36, 'more_1500': 0.36}}

In [114]:
def elbow_method(data, x_ax, y_ax):

    coords = []
    for i,row in data.iterrows():
        coords.append([row[x_ax], row[y_ax]])
    
    wcss = []
    max_clusters = len(data)
    for k in range(1,max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(coords)
        wcss.append(kmeans.inertia_)
    

    knee_locator = KneeLocator(range(1, max_clusters + 1), wcss, curve="convex", direction="decreasing")
    elbow_point = knee_locator.elbow
    
    # Minimum one cluster
    if elbow_point == None:
        elbow_point = 1

    return elbow_point
# Cluster data using weighted measure
def create_clusters(data, x_ax, y_ax, k, weight):

    coord_x = np.zeros(len(data))
    coord_y = np.zeros(len(data))

    for i, row in data.iterrows():
        coord_x[i] = data[x_ax][i]
        coord_y[i] = data[y_ax][i]
        
    X = np.array(list(zip(coord_x,coord_y))).reshape(len(coord_x),2)
    scaled_weights = (data[weight]/1000).astype(int)
    X_weighted = np.repeat(X, scaled_weights, axis=0)

    gmm = GaussianMixture(n_components=k).fit(X_weighted)

    centroids = gmm.means_.tolist()
    centroids = pd.DataFrame(centroids, columns=[x_ax, y_ax])
    centroids['id'] = pd.Series(centroids.index)
    labels = gmm.predict(X).tolist()

    return centroids, labels


# Distance matrix MCFII
def distance_matrix_II(url, source, sink, source_id, sink_id, source_lat, sink_lat, source_lon, sink_lon, transport_method):

    matrix = pd.DataFrame()

    if transport_method == 'pipe':

        for i, rsink in sink.iterrows():
            for j, rsource in source.iterrows():
                matrix.at[i,j] = geodesic((float(rsource[source_lat]),float(rsource[source_lon])), (float(rsink[sink_lat]),float(rsink[sink_lon]))).km


    elif transport_method == 'truck_ship':

        sink = sink.rename(columns={sink_lat:source_lat, sink_lon:source_lon})
        combined_df = pd.concat([source, sink], ignore_index=True)
        all_coords  = ";".join([f"{lon},{lat}" for lat, lon in zip(combined_df[source_lat], combined_df[source_lon])])

        params = {
            "sources": ";".join(map(str, range(len(source)))),
            "destinations": ";".join(map(str, range(len(source), len(source)+len(sink)))),
            "annotations": "distance"  # Request both
        }

        # Make the request
        response = requests.get(url + all_coords, params=params)
        data = response.json()

        # Truck matrix
        matrix = pd.DataFrame(data['distances'])

        # Add ship transportation (distance from final point)
        for col in range(len(matrix.columns)):
            matrix[col] = matrix[col] + data['destinations'][0]['distance']

        # Transpose and convert to km
        matrix = matrix.T/1000

    else:
        raise Exception("Error: Wrong transportation method")

    matrix = matrix.set_index(sink[sink_id])
    matrix = matrix.rename(columns=source[source_id])

    return matrix



# Cost matrix source-centr
def cost_matrix_source_centr(matrix, method, transport_cost, capture_cost):

    for col in matrix.columns:

        for id in matrix[col].index:

            if matrix.at[id,col]<180:
                matrix.at[id,col] = matrix.at[id,col]*float(transport_cost[method]['less_180'])

            elif matrix.at[id,col] >= 180 and matrix.at[id,col] < 500:
                matrix.at[id,col] = matrix.at[id,col]*float(transport_cost[method]['range_180_500'])
            
            elif matrix.at[id,col] >= 500 and matrix.at[id,col] < 750:
                matrix.at[id,col] = matrix.at[id,col]*float(transport_cost[method]['range_500_750'])

            elif matrix.at[id,col] >= 750 and matrix.at[id,col] < 1500:
                matrix.at[id,col] = matrix.at[id,col]*float(transport_cost[method]['range_750_1500'])

            else:
                matrix.at[id,col] = matrix.at[id,col]*float(transport_cost[method]['more_1500'])
            
    return matrix





# Cost matrix centr-sink
def cost_matrix_centr_sink(matrix, method, transport_cost, cluster_size):

    for col in matrix.columns:

        for id in matrix[col].index:

            if matrix.at[id,col]<180:
                matrix.at[id,col] = matrix.at[id,col]*float(transport_cost[method]['less_180'])/cluster_size[col]

            elif matrix.at[id,col] >= 180 and matrix.at[id,col] < 500:
                matrix.at[id,col] = matrix.at[id,col]*float(transport_cost[method]['range_180_500'])/cluster_size[col]
            
            elif matrix.at[id,col] >= 500 and matrix.at[id,col] < 750:
                matrix.at[id,col] = matrix.at[id,col]*float(transport_cost[method]['range_500_750'])/cluster_size[col]

            elif matrix.at[id,col] >= 750 and matrix.at[id,col] < 1500:
                matrix.at[id,col] = matrix.at[id,col]*float(transport_cost[method]['range_750_1500'])/cluster_size[col]

            else:
                matrix.at[id,col] = matrix.at[id,col]*float(transport_cost[method]['more_1500'])/cluster_size[col]

    return matrix


In [115]:
def network_optimization_klust_levelized(df_source, df_sink, df_cost_matrix, source_id, sink_id, source_capacity, sink_capacity, url, transport_method, transport_cost, emission_cost, capture_cost, quantity_cost_segments, stock_cost):

    # Run optimization for all nodes
    mcf_I_results = network_optimization_levelized(df_source, df_sink, df_cost_matrix, source_id, sink_id, source_capacity, sink_capacity, emission_cost, transport_method, quantity_cost_segments, capture_cost, stock_cost)

    # Extract nodes failed to connect
    unconnected_df = mcf_I_results[mcf_I_results['sink_id'] == "Atmosphere"]

    # Check if clusterization needed 
    if len(unconnected_df) == 0:
        # No need clustering
        return mcf_I_results
    else:
        # Need clustering
        df_source[source_id] = pd.Series(df_source[source_id].astype(str))
        df_sink[sink_id] = pd.Series(df_sink[sink_id].astype(str))

        df_source = df_source.rename(columns={source_id:f'source_{source_id}'})
        df_sink = df_sink.rename(columns={sink_id:f'sink_{sink_id}'})

        unconnected_df = pd.merge(unconnected_df, df_source, how='left', on='source_id')   
        unconnected_df = unconnected_df.rename(columns={'lat':'source_lat', 'lon':'source_lon'})    

        # Cluster the unconnected nodes
        el_point = elbow_method(unconnected_df, 'source_lat', 'source_lon')
        unconnected_df = unconnected_df.reset_index(drop=True)
        centr, labs = create_clusters(unconnected_df, 'source_lat', 'source_lon', el_point, 'co2_transported')

        # Merge centroids and unconndected points
        unconnected_df['cluster_lab'] = pd.Series(labs)
        centr = centr.rename(columns={'id':'cluster_lab','source_lat':'cluster_lat','source_lon':'cluster_lon'})
        unconnected_df = pd.merge(unconnected_df, centr, on='cluster_lab', how='left')

        # New distance and cost matrix

        # Count number of nodes per cluster
        cluster_size = unconnected_df.groupby('cluster_lab').size()
        # Distance source - centroid
        matrix_dist_source_centr = distance_matrix_II(url, unconnected_df, centr, 'source_id', 'cluster_lab', "source_lat", "cluster_lat", "source_lon", "cluster_lon", transport_method)
        # Cost source - centroid (individual)
        matrix_cost_source_centr = cost_matrix_source_centr(matrix_dist_source_centr, transport_method, transport_cost, capture_cost)
        # Distance centroid - sink
        matrix_dist_centr_sink = distance_matrix_II(url, centr, df_sink, 'cluster_lab', 'sink_id', 'cluster_lat', 'latitude', 'cluster_lon', 'longitude', transport_method)
        # Cost centroid - sink (shared)
        matrix_cost_centr_sink = cost_matrix_centr_sink(matrix_dist_centr_sink, transport_method, transport_cost, cluster_size)

        # General cost matrix setup
        matrix = pd.DataFrame(index=df_sink['sink_id'], columns=unconnected_df['source_id'])
        for i in matrix.index: #sinks
            for j in matrix.columns: #sources
                matrix.at[i,j] = matrix_cost_source_centr.loc[unconnected_df.set_index('source_id')['cluster_lab'].loc[j], j]
                matrix.at[i,j] = matrix.at[i,j] + matrix_cost_centr_sink.loc[i, unconnected_df.set_index('source_id')['cluster_lab'].loc[j]]
        atmosphere_row = pd.DataFrame({col:emission_cost for col in matrix.columns}, index=["Atmosphere"])
        matrix = pd.concat([matrix, atmosphere_row], axis=0)
        matrix = matrix.reset_index().rename(columns={'index':'Unnamed: 0'})

        # Run optimization for clustered nodes ()
        mcf_II_results = network_optimization_levelized(unconnected_df, df_sink, matrix, "source_id", "sink_id", source_capacity, sink_capacity, emission_cost, transport_method, quantity_cost_segments, capture_cost, stock_cost)


        # Merge results
        mcf_II_results['source_id'] = mcf_II_results['source_id'].astype(int)
        mcf_II_results = mcf_II_results.merge(unconnected_df[['source_id', 'cluster_lab', 'cluster_lat', 'cluster_lon']], on='source_id', how='left')

        # results = mcf_I_results.merge(mcf_II_results[['source_id','sink_id','cluster_lab','cluster_lat','cluster_lon']],
        #                               on='source_id',
        #                               how='left',
        #                               suffixes=('_I','_II'))
        
        # results['sink_id'] = results['sink_id_II'].combine_first(results['sink_id_I'])
        # cluster_update = ~results['sink_id_II'].isna()
        # results['cluster_lab'] = results['cluster_lab'].where(cluster_update, None)
        # results['cluster_lat'] = results['cluster_lat'].where(cluster_update, None)
        # results['cluster_lon'] = results['cluster_lon'].where(cluster_update, None)

        # results = results.drop(columns=['sink_id_I', 'sink_id_II'])


        mcf_I_results['source_id'] = mcf_I_results['source_id'].astype(int)
        merged = mcf_I_results.merge(mcf_II_results[['source_id', 'sink_id']], 
                  on='source_id', 
                  how='left',
                  suffixes=('_original', '_updated'))

        is_updated = (merged['sink_id_original'] == 'Atmosphere') & (merged['sink_id_updated'] != 'Atmosphere')

        merged = merged.merge(mcf_II_results[['source_id', 'cluster_lab', 'cluster_lat', 'cluster_lon']], 
                     on='source_id', 
                     how='left')
        
        results = mcf_I_results.copy()
        results['sink_id'] = merged['sink_id_updated'].combine_first(merged['sink_id_original'])
        results['cluster_lab'] = merged['cluster_lab'].where(is_updated, np.nan)
        results['cluster_lat'] = merged['cluster_lat'].where(is_updated, np.nan)
        results['cluster_lon'] = merged['cluster_lon'].where(is_updated, np.nan)

        return results

In [126]:
capture_cost=10
emission_cost=50

In [127]:
network_optimization_klust_levelized(source, sink, matrix, 'id', 'id', 'emission', 'sum_mid', url, 'truck_ship', transport_cost, emission_cost, capture_cost, quantity_cost_segments, stock_cost)



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/2d/n_vsfm_n1_l_wtnsx7bt2g300000gn/T/7def82b77e5741c5ad383f74a2085f46-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/2d/n_vsfm_n1_l_wtnsx7bt2g300000gn/T/7def82b77e5741c5ad383f74a2085f46-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5196 COLUMNS
At line 53014 RHS
At line 58206 BOUNDS
At line 75796 ENDATA
Problem MODEL has 5191 rows, 20130 columns and 27720 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-5191) rows, 0 (-20130) columns and 0 (-27720) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 8.3467125e+08
After Postsolve, objective 8.3467125e+08, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 834671245

,source_id,sink_id,co2_transported,cluster_lab,cluster_lat,cluster_lon
0,1897479,10402,2794440.400,NaN,NaN,NaN
1,1897478,10397,1036067.600,0.0,19.926654,-100.307018
2,32438922,10402,960096.780,NaN,NaN,NaN
3,1897504,10386,883825.440,NaN,NaN,NaN
4,32438914,10397,802542.670,0.0,19.926654,-100.307018
5,1897481,Atmosphere,802405.950,NaN,NaN,NaN
6,1897492,10402,784826.070,NaN,NaN,NaN
7,32438910,Atmosphere,784710.700,NaN,NaN,NaN
8,1897508,Atmosphere,757999.270,NaN,NaN,NaN
9,32438894,10392,720891.750,NaN,NaN,NaN


In [128]:
def update_results_with_cluster_connections(all_results, cluster_results, unconnected_df):
    """
    Updates the master results DataFrame with new connections from cluster optimization
    
    Args:
        all_results: pd.DataFrame - Master results from all previous iterations
        cluster_results: pd.DataFrame - Results from current cluster optimization
        unconnected_df: pd.DataFrame - Sources that were unconnected before this iteration
    
    Returns:
        pd.DataFrame - Updated results with new cluster connections
    """
    # Create a copy to avoid modifying the original
    updated_results = all_results.copy()
    
    # Mark which sources were processed in this clustering iteration
    processed_sources = unconnected_df['source_id'].unique()
    
    # First, set previously unconnected sources to 'pending' status
    updated_results.loc[
        updated_results['source_id'].isin(processed_sources) & 
        (updated_results['sink_id'] == "Atmosphere"),
        'sink_id'
    ] = "pending_cluster"
    
    # Prepare cluster connection data
    cluster_connections = cluster_results[
        (cluster_results['sink_id'] != "Atmosphere") & 
        (~cluster_results['sink_id'].str.startswith('cluster_'))
    ].copy()
    
    if len(cluster_connections) > 0:
        # These are direct connections from clusters to final sinks
        cluster_connections['cluster_level'] = 1
        updated_results = pd.concat([updated_results, cluster_connections], ignore_index=True)
    
    # Handle intra-cluster connections (source to centroid)
    intra_cluster = cluster_results[
        cluster_results['sink_id'].str.startswith('cluster_')
    ].copy()
    
    if len(intra_cluster) > 0:
        # Add cluster metadata
        intra_cluster = intra_cluster.merge(
            unconnected_df[['source_id', 'cluster_lab', 'cluster_lat', 'cluster_lon']],
            on='source_id',
            how='left'
        )
        intra_cluster['cluster_level'] = 1
        updated_results = pd.concat([updated_results, intra_cluster], ignore_index=True)
    
    # Clean up any remaining pending connections
    updated_results.loc[
        (updated_results['sink_id'] == "pending_cluster"),
        'sink_id'
    ] = "Atmosphere"
    
    # Add hierarchy tracking columns if they don't exist
    if 'cluster_hierarchy' not in updated_results.columns:
        updated_results['cluster_hierarchy'] = None
    
    # Update hierarchy information
    for _, row in cluster_results.iterrows():
        source = row['source_id']
        sink = row['sink_id']
        
        if sink != "Atmosphere":
            if sink.startswith('cluster_'):
                # Connection to cluster centroid
                hierarchy = f"L1:{sink}"
            else:
                # Direct connection from cluster to final sink
                hierarchy = "L1:direct"
            
            # Update hierarchy info
            mask = (updated_results['source_id'] == source) & \
                   (updated_results['sink_id'] == sink)
            updated_results.loc[mask, 'cluster_hierarchy'] = hierarchy
    
    return updated_results

In [145]:
def elbow_method(coords, weights=None):
    """Calculate optimal number of clusters using elbow method with weighted points"""
    # Ensure coordinates are 2D
    coords = np.array(coords)
    if coords.ndim == 1:
        coords = coords.reshape(-1, 1)
    
    # Prepare weighted coordinates if weights are provided
    if weights is not None:
        weights = np.array(weights)
        scaled_weights = (weights / weights.min()).astype(int)
        coords_weighted = np.repeat(coords, scaled_weights, axis=0)
    else:
        coords_weighted = coords
    
    wcss = []
    max_clusters = min(10, len(np.unique(coords_weighted, axis=0)))
    
    for k in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(coords_weighted)
        wcss.append(kmeans.inertia_)
    
    # Find elbow point
    if len(wcss) > 1:
        knee_locator = KneeLocator(
            range(1, max_clusters + 1), 
            wcss, 
            curve="convex", 
            direction="decreasing"
        )
        optimal_clusters = knee_locator.elbow or 1
    else:
        optimal_clusters = 1
    
    return optimal_clusters

def create_clusters(coords, k, weights=None):
    """Create clusters using Gaussian Mixture Model with weights"""
    coords = np.array(coords)
    if coords.ndim == 1:
        coords = coords.reshape(-1, 1)
    
    if weights is not None:
        weights = np.array(weights)
        scaled_weights = (weights / weights.min()).astype(int)
        coords_weighted = np.repeat(coords, scaled_weights, axis=0)
        gmm = GaussianMixture(n_components=k).fit(coords_weighted)
        labels = gmm.predict(coords)  # Predict on original coordinates
    else:
        gmm = GaussianMixture(n_components=k).fit(coords)
        labels = gmm.predict(coords)
    
    # Calculate centers from weighted points
    if weights is not None:
        weighted_coords = np.concatenate([coords, weights.reshape(-1, 1)], axis=1)
        centers = np.array([
            np.average(
                weighted_coords[labels == i, :-1], 
                axis=0, 
                weights=weighted_coords[labels == i, -1]
            ) for i in range(k)
        ])
    else:
        centers = gmm.means_
    
    return centers, labels

In [146]:
def network_optimization_iterative_clustering(df_source, df_sink, df_cost_matrix, source_id, sink_id, 
                                            source_capacity, sink_capacity, url, transport_method, 
                                            transport_cost, emission_cost, capture_cost, 
                                            quantity_cost_segments, stock_cost):
    
    # First optimization - direct connections
    results = network_optimization_levelized(
        df_source.copy(),
        df_sink.copy(),
        df_cost_matrix.copy(),
        source_id,
        sink_id,
        source_capacity,
        sink_capacity,
        emission_cost,
        transport_method,
        quantity_cost_segments,
        capture_cost,
        stock_cost
    )
    
    # Identify unconnected nodes
    unconnected_df = results[results['sink_id'] == "Atmosphere"].copy()
    
    iteration = 1
    prev_centroids = pd.DataFrame()

    while len(unconnected_df) >= 2:
        print(f"\nStarting clustering iteration {iteration} with {len(unconnected_df)} unconnected nodes")
        
        try:
            # 1. Prepare source data with proper column handling
            df_source_prep = df_source.copy()
            df_source_prep[source_id] = df_source_prep[source_id].astype(str)
            unconnected_df['source_id'] = unconnected_df['source_id'].astype(str)
            
            # 2. Safer merge operation
            merge_columns = [source_id, 'lat', 'lon']
            unconnected_df = unconnected_df.merge(
                df_source_prep[merge_columns],
                how='left',
                left_on='source_id',
                right_on=source_id,
                suffixes=('', '_src')
            )
            
            # Clean up merged columns
            if f'{source_id}_src' in unconnected_df.columns:
                unconnected_df = unconnected_df.drop(columns=[f'{source_id}_src'])
            
            unconnected_df = unconnected_df.rename(columns={
                'lat': 'source_lat',
                'lon': 'source_lon'
            })
            
            # 3. Cluster with transport-aware weights
            coords = unconnected_df[['source_lat', 'source_lon']].values
            weights = unconnected_df['co2_transported'].values
            
            el_point = elbow_method(coords, weights)
            centr, labs = create_clusters(coords, el_point, weights)
            
            # 4. Prepare cluster data with transport attributes
            unconnected_df['cluster_lab'] = [str(x) for x in labs]
            centr = pd.DataFrame({
                'cluster_lab': [str(i) for i in range(len(centr))],
                'cluster_lat': centr[:, 0],
                'cluster_lon': centr[:, 1],
                'total_co2': unconnected_df.groupby('cluster_lab')['co2_transported'].sum()
            }).reset_index(drop=True)
            
            # 5. Calculate source-to-centroid transport costs
            matrix_dist_source_centr = distance_matrix_II(
                url,
                unconnected_df.rename(columns={
                    'source_id': 'from_id',
                    'source_lat': 'from_lat',
                    'source_lon': 'from_lon'
                }),
                centr.rename(columns={
                    'cluster_lab': 'to_id',
                    'cluster_lat': 'to_lat',
                    'cluster_lon': 'to_lon'
                }),
                'from_id', 'to_id',
                "from_lat", "to_lat",
                "from_lon", "to_lon",
                transport_method
            )
            
            # Apply transport cost rates
            matrix_cost_source_centr = cost_matrix_source_centr(
                matrix_dist_source_centr,
                transport_method,
                transport_cost,
                capture_cost
            )
            
            # 6. Create virtual sources
            cluster_sources = centr.copy()
            cluster_sources[source_id] = 'cluster_' + cluster_sources['cluster_lab']
            cluster_sources[source_capacity] = cluster_sources['total_co2']
            
            # 7. Prepare sinks (including previous centroids)
            if iteration == 1:
                all_sinks = df_sink.copy()
            else:
                prev_centroids[sink_id] = 'cluster_' + prev_centroids['cluster_lab']
                prev_centroids = prev_centroids.rename(columns={
                    'cluster_lat': 'latitude',
                    'cluster_lon': 'longitude'
                })
                all_sinks = pd.concat([df_sink, prev_centroids], ignore_index=True)
            
            all_sinks[sink_id] = all_sinks[sink_id].astype(str)
            
            # 8. Calculate centroid-to-sink transport costs with shared costs
            matrix_dist_centr_sink = distance_matrix_II(
                url,
                centr.rename(columns={
                    'cluster_lab': 'from_id',
                    'cluster_lat': 'from_lat',
                    'cluster_lon': 'from_lon'
                }),
                all_sinks.rename(columns={
                    sink_id: 'to_id',
                    'latitude': 'to_lat',
                    'longitude': 'to_lon'
                }),
                'from_id', 'to_id',
                "from_lat", "to_lat",
                "from_lon", "to_lon",
                transport_method
            )
            
            cluster_size = unconnected_df.groupby('cluster_lab').size()
            matrix_cost_centr_sink = cost_matrix_centr_sink(
                matrix_dist_centr_sink,
                transport_method,
                transport_cost,
                cluster_size
            )
            
            # 9. Build combined cost matrix
            combined_matrix = pd.DataFrame(columns=['Unnamed: 0'] + unconnected_df['source_id'].tolist())
            combined_matrix['Unnamed: 0'] = all_sinks[sink_id].tolist() + ['Atmosphere']
            
            for idx, sink_row in combined_matrix.iterrows():
                sink_id_val = str(sink_row['Unnamed: 0'])
                for source in unconnected_df['source_id']:
                    source = str(source)
                    cluster_id = unconnected_df.loc[unconnected_df['source_id'] == source, 'cluster_lab'].values[0]
                    
                    # Source to centroid cost
                    source_centr_cost = matrix_cost_source_centr.at[cluster_id, source]
                    
                    if sink_id_val == 'Atmosphere':
                        combined_matrix.at[idx, source] = emission_cost
                    else:
                        # Centroid to sink cost
                        centroid_sink_cost = matrix_cost_centr_sink.at[sink_id_val, cluster_id]
                        combined_matrix.at[idx, source] = source_centr_cost + centroid_sink_cost
            
            # 10. Run optimization
            cluster_results = network_optimization_levelized(
                unconnected_df.rename(columns={'source_id': source_id}),
                all_sinks.rename(columns={sink_id: sink_id}),
                combined_matrix,
                source_id,
                sink_id,
                source_capacity,
                sink_capacity,
                emission_cost,
                transport_method,
                quantity_cost_segments,
                capture_cost,
                stock_cost
            )
            
            # Update results
            results = update_results_with_cluster_connections(results, cluster_results, unconnected_df)
            prev_centroids = centr.copy()
            unconnected_df = results[results['sink_id'] == "Atmosphere"].copy()
            iteration += 1
            
        except Exception as e:
            print(f"Error in iteration {iteration}: {str(e)}")
            print("Current columns in unconnected_df:", unconnected_df.columns.tolist())
            break
    
    return results

In [147]:
results = network_optimization_iterative_clustering(source, sink, matrix, 'id', 'id', 
                                            'emission', 'sum_mid', url, 'truck_ship', 
                                            transport_cost, emission_cost, capture_cost, 
                                            quantity_cost_segments, stock_cost)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/2d/n_vsfm_n1_l_wtnsx7bt2g300000gn/T/15d796d784654e5a85be0d0f8b49bb8c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/2d/n_vsfm_n1_l_wtnsx7bt2g300000gn/T/15d796d784654e5a85be0d0f8b49bb8c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5196 COLUMNS
At line 53014 RHS
At line 58206 BOUNDS
At line 75796 ENDATA
Problem MODEL has 5191 rows, 20130 columns and 27720 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-5191) rows, 0 (-20130) columns and 0 (-27720) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 8.3467125e+08
After Postsolve, objective 8.3467125e+08, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 834671245

In [140]:
results

,source_id,sink_id,co2_transported
0,1897479,10402,2794440.400
1,1897478,Atmosphere,1036067.600
2,32438922,10402,960096.780
3,1897504,10386,883825.440
4,32438914,Atmosphere,802542.670
5,1897481,Atmosphere,802405.950
6,1897492,10402,784826.070
7,32438910,Atmosphere,784710.700
8,1897508,Atmosphere,757999.270
9,32438894,10392,720891.750


In [148]:
def network_optimization_iterative_clustering(df_source, df_sink, df_cost_matrix, source_id, sink_id, 
                                            source_capacity, sink_capacity, url, transport_method, 
                                            transport_cost, emission_cost, capture_cost, 
                                            quantity_cost_segments, stock_cost):
    
    # Make copies to avoid modifying original dataframes
    df_source = df_source.copy()
    df_sink = df_sink.copy()
    df_cost_matrix = df_cost_matrix.copy()
    
    # First optimization - direct connections
    results = network_optimization_levelized(
        df_source,
        df_sink,
        df_cost_matrix,
        source_id,
        sink_id,
        source_capacity,
        sink_capacity,
        emission_cost,
        transport_method,
        quantity_cost_segments,
        capture_cost,
        stock_cost
    )
    
    # Identify unconnected nodes
    unconnected_df = results[results['sink_id'] == "Atmosphere"].copy()
    
    iteration = 1
    prev_centroids = pd.DataFrame()

    while len(unconnected_df) >= 2:
        print(f"\nStarting clustering iteration {iteration} with {len(unconnected_df)} unconnected nodes")
        
        try:
            # 1. Prepare source data - handle column names carefully
            required_source_columns = [source_id, 'lat', 'lon']
            if not all(col in df_source.columns for col in required_source_columns):
                raise ValueError(f"Source dataframe missing required columns. Needs: {required_source_columns}")
            
            # Clean column names before merge
            unconnected_df = unconnected_df.rename(columns={
                'source_id': 'tmp_source_id'
            })
            
            # 2. Merge with source data
            unconnected_df = pd.merge(
                unconnected_df,
                df_source[required_source_columns],
                how='left',
                left_on='tmp_source_id',
                right_on=source_id,
                suffixes=('', '_src')
            ).rename(columns={
                'lat': 'source_lat',
                'lon': 'source_lon',
                'tmp_source_id': 'source_id'
            })
            
            # Clean up merged columns
            if f'{source_id}_src' in unconnected_df.columns:
                unconnected_df = unconnected_df.drop(columns=[f'{source_id}_src'])
            
            # 3. Cluster the unconnected nodes with transport-aware weights
            coords = unconnected_df[['source_lat', 'source_lon']].values
            weights = unconnected_df['co2_transported'].values
            
            # Calculate optimal clusters considering transport costs
            optimal_clusters = calculate_optimal_clusters(coords, weights)
            centr, labs = create_weighted_clusters(coords, optimal_clusters, weights)
            
            # 4. Prepare cluster data with transport attributes
            unconnected_df['cluster_lab'] = [str(x) for x in labs]
            centr_df = pd.DataFrame({
                'cluster_lab': [str(i) for i in range(len(centr))],
                'cluster_lat': centr[:, 0],
                'cluster_lon': centr[:, 1],
                'total_co2': unconnected_df.groupby('cluster_lab')['co2_transported'].sum()
            }).reset_index(drop=True)
            
            # 5. Calculate source-to-centroid transport costs
            print("Calculating source-to-centroid transport costs...")
            matrix_dist_source_centr = distance_matrix_II(
                url,
                unconnected_df.rename(columns={
                    'source_id': 'from_id',
                    'source_lat': 'from_lat',
                    'source_lon': 'from_lon'
                }),
                centr_df.rename(columns={
                    'cluster_lab': 'to_id',
                    'cluster_lat': 'to_lat',
                    'cluster_lon': 'to_lon'
                }),
                'from_id', 'to_id',
                "from_lat", "to_lat",
                "from_lon", "to_lon",
                transport_method
            )
            
            # Apply transport cost rates
            matrix_cost_source_centr = cost_matrix_source_centr(
                matrix_dist_source_centr,
                transport_method,
                transport_cost,
                capture_cost
            )
            
            # [Rest of the implementation remains the same...]
            
            # Update results and prepare for next iteration
            results = update_results_with_cluster_connections(results, cluster_results, unconnected_df)
            prev_centroids = centr_df.copy()
            unconnected_df = results[results['sink_id'] == "Atmosphere"].copy()
            iteration += 1
            
        except Exception as e:
            print(f"Error in iteration {iteration}: {str(e)}")
            print("Current columns in unconnected_df:", unconnected_df.columns.tolist())
            print("Current centroids:", centr_df.columns if 'centr_df' in locals() else "Not created")
            break
    
    return results


def calculate_optimal_clusters(coords, weights=None, max_clusters=10):
    """Robust elbow method with weighted points"""
    from sklearn.cluster import KMeans
    from kneed import KneeLocator
    import numpy as np
    
    coords = np.array(coords)
    if coords.ndim == 1:
        coords = coords.reshape(-1, 1)
    
    # Prepare weighted dataset if weights are provided
    if weights is not None:
        weights = np.array(weights)
        scaled_weights = (weights / weights.min()).astype(int)
        coords_weighted = np.repeat(coords, scaled_weights, axis=0)
    else:
        coords_weighted = coords
    
    max_clusters = min(max_clusters, len(np.unique(coords_weighted, axis=0)))
    if max_clusters <= 1:
        return 1
    
    wcss = []
    for k in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=0).fit(coords_weighted)
        wcss.append(kmeans.inertia_)
    
    if len(wcss) > 1:
        knee_locator = KneeLocator(
            range(1, max_clusters + 1),
            wcss,
            curve='convex',
            direction='decreasing'
        )
        return knee_locator.elbow or 1
    return 1


def create_weighted_clusters(coords, n_clusters, weights=None):
    """Create clusters using GMM with proper weight handling"""
    from sklearn.mixture import GaussianMixture
    import numpy as np
    
    coords = np.array(coords)
    if coords.ndim == 1:
        coords = coords.reshape(-1, 1)
    
    if weights is not None:
        weights = np.array(weights)
        scaled_weights = (weights / weights.min()).astype(int)
        coords_weighted = np.repeat(coords, scaled_weights, axis=0)
        gmm = GaussianMixture(n_components=n_clusters).fit(coords_weighted)
    else:
        gmm = GaussianMixture(n_components=n_clusters).fit(coords)
    
    labels = gmm.predict(coords)
    centers = gmm.means_
    
    return centers, labels

In [149]:
results = network_optimization_iterative_clustering(source, sink, matrix, 'id', 'id', 
                                            'emission', 'sum_mid', url, 'truck_ship', 
                                            transport_cost, emission_cost, capture_cost, 
                                            quantity_cost_segments, stock_cost)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/2d/n_vsfm_n1_l_wtnsx7bt2g300000gn/T/210a3a684c794eacb6d3b5e4994f8529-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/2d/n_vsfm_n1_l_wtnsx7bt2g300000gn/T/210a3a684c794eacb6d3b5e4994f8529-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5196 COLUMNS
At line 53014 RHS
At line 58206 BOUNDS
At line 75796 ENDATA
Problem MODEL has 5191 rows, 20130 columns and 27720 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-5191) rows, 0 (-20130) columns and 0 (-27720) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 8.3467125e+08
After Postsolve, objective 8.3467125e+08, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 834671245

In [187]:
import pulp
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from math import radians, sin, cos, sqrt, atan2

def haversine(coord1, coord2):
    """Calculate Haversine distance between two coordinates"""
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = (sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2)
    return R * 2 * atan2(sqrt(a), sqrt(1-a))

def network_optimization_levelized(df_source, df_sink, df_cost_matrix, source_id, sink_id,
                                 source_capacity, sink_capacity, emission_cost, transport_method,
                                 quantity_cost_segments, capture_cost, stock_cost):
    """Core network optimization function"""
    try:
        # Convert IDs to strings
        df_source[source_id] = df_source[source_id].astype(str)
        df_sink[sink_id] = df_sink[sink_id].astype(str)
        
        # Prepare cost matrix
        if 'Unnamed: 0' in df_cost_matrix.columns:
            df_cost_matrix = df_cost_matrix.rename(columns={'Unnamed: 0': sink_id})
        transport_cost = df_cost_matrix.set_index(sink_id)

        # Initialize problem
        network = pulp.LpProblem("Network_Optimization", pulp.LpMinimize)
        
        # Nodes and arcs
        sources = df_source[source_id].unique()
        sinks = df_sink[sink_id].unique().tolist()
        
        # Add atmosphere as special sink
        sinks.append('Atmosphere')
        
        arcs = []
        arc_capacities = {}
        for src in sources:
            for snk in sinks:
                arc = (f"src_{src}", f"snk_{snk}")
                arcs.append(arc)
                # Use large but finite capacity
                arc_capacities[arc] = 1e12 if snk == 'Atmosphere' else 10_000_000

        # Variables
        flow_vars = pulp.LpVariable.dicts("Flow", arcs, 0, None, pulp.LpContinuous)
        
        # Cost segments
        cost_segments = {}
        for src in sources:
            for snk in sinks:
                if snk == 'Atmosphere':
                    cost_segments[(f"src_{src}", f"snk_{snk}")] = [(0, 1e12, emission_cost)]
                else:
                    try:
                        cost = transport_cost.at[snk, src]
                        cost_segments[(f"src_{src}", f"snk_{snk}")] = [
                            (0, 50000, quantity_cost_segments[transport_method][(0, 50000)] * cost),
                            (50000, 100000, quantity_cost_segments[transport_method][(50000, 100000)] * cost),
                            (100000, 250000, quantity_cost_segments[transport_method][(100000, 250000)] * cost),
                            (250000, 500000, quantity_cost_segments[transport_method][(250000, 500000)] * cost),
                            (500000, 1000000, quantity_cost_segments[transport_method][(500000, 1000000)] * cost),
                            (1000000, 2000000, quantity_cost_segments[transport_method][(1000000, 2000000)] * cost),
                            (2000000, 1e12, quantity_cost_segments[transport_method][(2000000, 999999999)] * cost)
                        ]
                    except KeyError:
                        print(f"Missing cost for source {src} to sink {snk}")
                        continue

        # Segment variables
        segment_vars = {}
        for arc in arcs:
            if arc not in cost_segments:
                continue
            segment_vars[arc] = []
            for i, (start, end, slope) in enumerate(cost_segments[arc]):
                upper_bound = end - start
                var = pulp.LpVariable(f"Seg_{arc[0]}_{arc[1]}_{i}", 0, upper_bound, pulp.LpContinuous)
                segment_vars[arc].append((var, slope))

        # Objective function
        network += (
            pulp.lpSum(flow_vars[arc] * capture_cost for arc in arcs if arc[1] != 'snk_Atmosphere') +
            pulp.lpSum(var * slope for arc in arcs for (var, slope) in segment_vars.get(arc, [])) +
            pulp.lpSum(flow_vars[arc] * stock_cost for arc in arcs if arc[1] != 'snk_Atmosphere') +
            pulp.lpSum(flow_vars[arc] * emission_cost for arc in arcs if arc[1] == 'snk_Atmosphere')
        ), "Total_Cost"

        # Constraints
        for _, src_row in df_source.iterrows():
            src = str(src_row[source_id])
            network += (
                pulp.lpSum(flow_vars.get((f"src_{src}", f"snk_{snk}"), 0)
                for snk in sinks
            ) == src_row[source_capacity], f"Source_{src}_outflow")

        for _, snk_row in df_sink.iterrows():
            snk = str(snk_row[sink_id])
            network += (
                pulp.lpSum(flow_vars.get((f"src_{src}", f"snk_{snk}"), 0)
                for src in sources
            ) <= snk_row[sink_capacity], f"Sink_{snk}_inflow")

        for arc in arcs:
            if arc in segment_vars:
                network += flow_vars[arc] == pulp.lpSum(var for (var, _) in segment_vars[arc])
                network += flow_vars[arc] <= arc_capacities[arc]

        # Solve
        network.solve(pulp.PULP_CBC_CMD(msg=False))
        
        # Process results
        results = []
        for arc in arcs:
            if arc in flow_vars and flow_vars[arc].varValue is not None and flow_vars[arc].varValue > 1e-6:
                src = arc[0].replace("src_", "")
                snk = arc[1].replace("snk_", "")
                results.append({
                    'source_id': src,
                    'sink_id': snk,
                    'co2_transported': flow_vars[arc].varValue,
                    'path': f"{src}→{snk}"
                })
        
        # Return empty DataFrame with correct columns if no results
        if not results:
            return pd.DataFrame(columns=['source_id', 'sink_id', 'co2_transported', 'path'])
        
        return pd.DataFrame(results)
    
    except Exception as e:
        print(f"Optimization failed: {str(e)}")
        return pd.DataFrame(columns=['source_id', 'sink_id', 'co2_transported', 'path'])

def iterative_clustered_network_optimization(
    df_source, df_sink, df_cost_matrix, source_id, sink_id,
    source_capacity, sink_capacity, emission_cost, transport_method,
    quantity_cost_segments, capture_cost, stock_cost, max_iter=5
):
    """Main iterative optimization with clustering"""
    try:
        # Initialize
        df_source = df_source.copy()
        df_sink = df_sink.copy()
        df_cost_matrix = df_cost_matrix.copy()
        
        all_results = []
        cluster_history = {}
        
        for iteration in range(max_iter):
            # Run optimization
            current_results = network_optimization_levelized(
                df_source, df_sink, df_cost_matrix, source_id, sink_id,
                source_capacity, sink_capacity, emission_cost, transport_method,
                quantity_cost_segments, capture_cost, stock_cost
            )
            
            # Track connected sources
            connected = set(current_results['source_id'])
            unconnected = set(df_source[source_id].astype(str)) - connected
            
            if not unconnected:
                break
                
            # Cluster unconnected sources
            X = df_source[df_source[source_id].astype(str).isin(unconnected)]
            if len(X) < 2:
                break
                
            # Weighted clustering
            k = min(3, len(X))
            kmeans = KMeans(n_clusters=k, random_state=42)
            weights = X[source_capacity] / X[source_capacity].sum()
            kmeans.fit(X[['lat', 'lon']], sample_weight=weights)
            
            # Create centroids
            centroids = X.groupby(kmeans.labels_).agg({
                'lat': 'mean',
                'lon': 'mean',
                source_id: lambda x: f"centroid_{iteration}_" + "_".join(sorted(x.astype(str)))
            }).reset_index()
            
            # Update network
            for _, centroid in centroids.iterrows():
                centroid_id = centroid[source_id]
                if centroid_id in cluster_history:
                    continue
                    
                cluster_history[centroid_id] = (centroid['lat'], centroid['lon'])
                
                # Add centroid to sinks
                df_sink = pd.concat([
                    df_sink,
                    pd.DataFrame([{
                        sink_id: centroid_id,
                        'latitude': centroid['lat'],
                        'longitude': centroid['lon'],
                        'sum_mid': float('inf'),
                        'site_name': f"Cluster_{iteration}"
                    }])
                ], ignore_index=True)
                
                # Update cost matrix
                cluster_members = X[kmeans.labels_ == _][source_id].astype(str)
                for src in cluster_members:
                    # Individual connection to centroid
                    distance = haversine(
                        (df_source[df_source[source_id] == src]['lat'].values[0],
                        df_source[df_source[source_id] == src]['lon'].values[0]),
                        (centroid['lat'], centroid['lon'])
                    )
                    df_cost_matrix.loc[df_cost_matrix[sink_id] == centroid_id, src] = distance
                    
                    # Shared connection from centroid to sinks
                    for _, sink_row in df_sink[df_sink[sink_id] != centroid_id].iterrows():
                        original_cost = df_cost_matrix.loc[
                            df_cost_matrix[sink_id] == sink_row[sink_id], src].values[0]
                        shared_cost = original_cost / len(cluster_members)
                        df_cost_matrix.loc[
                            df_cost_matrix[sink_id] == sink_row[sink_id], centroid_id] = shared_cost

            # Store results with path info
            for _, row in current_results.iterrows():
                if row['sink_id'].startswith('centroid_'):
                    centroid = row['sink_id']
                    downstream = current_results[current_results['source_id'] == centroid]
                    for _, ds_row in downstream.iterrows():
                        all_results.append({
                            'source_id': row['source_id'],
                            'sink_id': ds_row['sink_id'],
                            'co2_transported': row['co2_transported'],
                            'path': f"{row['source_id']}→{centroid}→{ds_row['sink_id']}"
                        })
                else:
                    all_results.append(row.to_dict())

        return pd.DataFrame(all_results)
    
    except Exception as e:
        print(f"Iterative optimization failed: {str(e)}")
        return pd.DataFrame(columns=['source_id', 'sink_id', 'co2_transported', 'path'])


In [181]:
import pandas as pd
import numpy as np

# Example Data Preparation
def prepare_example_data():
    # Example Sources (Cement Plants)
    source_data = {
        'id': ['1897479', '1897478', '32438922', '1897504', '32438914'],
        'name': ['CEMEX Mexico SA de CV Puebla 1', 
                'CEMEX Mexico SA de CV Hidalgo 1',
                'Holcim Mexico SA de CV Veracruz 1',
                'Holcim Mexico SA de CV Tabasco 1',
                'Cementos Fortaleza Hidalgo 2'],
        'emission': [2.794440e+06, 1.036068e+06, 9.600968e+05, 8.838254e+05, 8.025427e+05],
        'lat': [18.967968, 19.996589, 18.857600, 17.651499, 20.011934],
        'lon': [-97.961858, -99.210844, -97.041208, -92.443296, -99.178466]
    }
    df_source = pd.DataFrame(source_data)

    # Example Sinks (Storage Sites)
    sink_data = {
        'id': ['10346', '10347', '10348', '10349', '10350'],
        'sum_mid': [4.300000e+08, 1.040000e+09, 1.920000e+09, 1.970000e+09, 5.400000e+08],
        'latitude': [25.642778, 25.664722, 25.758611, 25.779444, 25.809444],
        'longitude': [-99.071389, -98.954444, -98.580000, -98.438611, -97.863611],
        'site_name': ['Burgos B1A', 'Burgos B1B', 'Burgos B1C', 'Burgos B1D', 'Burgos B1E']
    }
    df_sink = pd.DataFrame(sink_data)

    # Example Cost Matrix (Distances between sources and sinks)
    cost_matrix_data = {
        'Unnamed: 0': ['10346', '10347', '10348', '10349', '10350'],
        '1897479': [345.817296, 344.745828, 348.487164, 340.211268, 336.485808],
        '1897478': [332.455428, 329.412024, 333.153360, 324.877428, 321.152004],
        '32438922': [373.947948, 372.876480, 376.617816, 368.341884, 364.616460],
        '1897504': [521.441244, 520.369740, 524.111148, 515.835180, 512.109720],
        '32438914': [331.633836, 329.152284, 332.893620, 324.617688, 320.892264]
    }
    df_cost_matrix = pd.DataFrame(cost_matrix_data)

    return df_source, df_sink, df_cost_matrix



In [182]:
# Prepare data (with string IDs)
df_source, df_sink, df_cost_matrix = prepare_example_data()

# Run optimization
results = iterative_clustered_network_optimization(
    df_source=df_source,
    df_sink=df_sink,
    df_cost_matrix=df_cost_matrix,
    source_id='id',
    sink_id='id',
    source_capacity='emission',
    sink_capacity='sum_mid',
    emission_cost=50,
    transport_method='pipe',
    quantity_cost_segments=quantity_cost_segments,
    capture_cost=30,
    stock_cost=10,
    max_iter=5
)

print(results[['source_id', 'sink_id', 'co2_transported', 'path']])

KeyError: "None of [Index(['source_id', 'sink_id', 'co2_transported', 'path'], dtype='object')] are in the [columns]"

In [188]:
from math import radians, sin, cos, sqrt, atan2

def haversine(coord1, coord2):
    """Calculate Haversine distance between two lat/lon coordinates."""
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth radius in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))


In [189]:
import pulp
import pandas as pd

def network_optimization_levelized(df_source, df_sink, df_cost_matrix, source_id, sink_id,
                                    source_capacity, sink_capacity, emission_cost, transport_method,
                                    quantity_cost_segments, capture_cost, stock_cost):
    """Solve the network optimization with levelized cost segmentation."""
    try:
        df_source[source_id] = df_source[source_id].astype(str)
        df_sink[sink_id] = df_sink[sink_id].astype(str)

        if 'Unnamed: 0' in df_cost_matrix.columns:
            df_cost_matrix = df_cost_matrix.rename(columns={'Unnamed: 0': sink_id})
        transport_cost = df_cost_matrix.set_index(sink_id)

        network = pulp.LpProblem("Network_Optimization", pulp.LpMinimize)

        sources = df_source[source_id].unique()
        sinks = df_sink[sink_id].unique().tolist()
        sinks.append('Atmosphere')  # Add virtual sink

        arcs = [(f"src_{s}", f"snk_{t}") for s in sources for t in sinks]
        arc_capacities = {(a, b): 1e12 if 'Atmosphere' in b else 10_000_000 for a, b in arcs}

        flow_vars = pulp.LpVariable.dicts("Flow", arcs, 0, None, pulp.LpContinuous)

        cost_segments = {}
        for s in sources:
            for t in sinks:
                arc = (f"src_{s}", f"snk_{t}")
                if 'Atmosphere' in t:
                    cost_segments[arc] = [(0, 1e12, emission_cost)]
                    continue
                try:
                    base_cost = transport_cost.at[t, s]
                    cost_segments[arc] = [
                        (start, end, quantity_cost_segments[transport_method][(start, end)] * base_cost)
                        for (start, end) in quantity_cost_segments[transport_method]
                    ]
                except KeyError:
                    continue

        segment_vars = {}
        for arc, segments in cost_segments.items():
            segment_vars[arc] = []
            for i, (start, end, slope) in enumerate(segments):
                var = pulp.LpVariable(f"Seg_{arc[0]}_{arc[1]}_{i}", 0, end - start, pulp.LpContinuous)
                segment_vars[arc].append((var, slope))

        network += (
            pulp.lpSum(flow_vars[arc] * capture_cost for arc in arcs if 'Atmosphere' not in arc[1]) +
            pulp.lpSum(var * slope for arc in arcs for var, slope in segment_vars.get(arc, [])) +
            pulp.lpSum(flow_vars[arc] * stock_cost for arc in arcs if 'Atmosphere' not in arc[1]) +
            pulp.lpSum(flow_vars[arc] * emission_cost for arc in arcs if 'Atmosphere' in arc[1])
        ), "Total_Cost"

        for _, row in df_source.iterrows():
            src = str(row[source_id])
            network += pulp.lpSum(flow_vars.get((f"src_{src}", f"snk_{snk}"), 0) for snk in sinks) == row[source_capacity]

        for _, row in df_sink.iterrows():
            snk = str(row[sink_id])
            network += pulp.lpSum(flow_vars.get((f"src_{src}", f"snk_{snk}"), 0) for src in sources) <= row[sink_capacity]

        for arc in arcs:
            if arc in segment_vars:
                network += flow_vars[arc] == pulp.lpSum(var for var, _ in segment_vars[arc])
            network += flow_vars[arc] <= arc_capacities[arc]

        network.solve(pulp.PULP_CBC_CMD(msg=False))

        results = []
        for arc in arcs:
            value = flow_vars[arc].varValue
            if value and value > 1e-6:
                results.append({
                    'source_id': arc[0].replace("src_", ""),
                    'sink_id': arc[1].replace("snk_", ""),
                    'co2_transported': value,
                    'path': f"{arc[0].replace('src_', '')}→{arc[1].replace('snk_', '')}"
                })

        return pd.DataFrame(results, columns=['source_id', 'sink_id', 'co2_transported', 'path'])

    except Exception as e:
        print(f"Optimization failed: {e}")
        return pd.DataFrame(columns=['source_id', 'sink_id', 'co2_transported', 'path'])


In [191]:
from sklearn.cluster import KMeans

def iterative_clustered_network_optimization(df_source, df_sink, df_cost_matrix,
                                             source_id, sink_id, source_capacity, sink_capacity,
                                             emission_cost, transport_method, quantity_cost_segments,
                                             capture_cost, stock_cost, max_iter=5):
    """Run iterative optimization with clustering of unconnected sources."""
    try:
        df_source = df_source.copy()
        df_sink = df_sink.copy()
        df_cost_matrix = df_cost_matrix.copy()

        all_results = []
        cluster_history = {}

        for iteration in range(max_iter):
            current_results = network_optimization_levelized(
                df_source, df_sink, df_cost_matrix, source_id, sink_id,
                source_capacity, sink_capacity, emission_cost, transport_method,
                quantity_cost_segments, capture_cost, stock_cost
            )

            connected = set(current_results['source_id'])
            unconnected = set(df_source[source_id].astype(str)) - connected

            if not unconnected or len(unconnected) < 2:
                break

            X = df_source[df_source[source_id].astype(str).isin(unconnected)]
            k = min(3, len(X))
            weights = X[source_capacity] / X[source_capacity].sum()
            kmeans = KMeans(n_clusters=k, random_state=42)
            kmeans.fit(X[['lat', 'lon']], sample_weight=weights)

            centroids = X.copy()
            centroids['cluster'] = kmeans.labels_
            for cluster_id, group in centroids.groupby('cluster'):
                members = group[source_id].astype(str).tolist()
                centroid_lat = group['lat'].mean()
                centroid_lon = group['lon'].mean()
                centroid_id = f"centroid_{iteration}_{'_'.join(sorted(members))}"

                if centroid_id in cluster_history:
                    continue
                cluster_history[centroid_id] = (centroid_lat, centroid_lon)

                df_sink = pd.concat([
                    df_sink,
                    pd.DataFrame([{
                        sink_id: centroid_id,
                        'latitude': centroid_lat,
                        'longitude': centroid_lon,
                        'sum_mid': float('inf'),
                        'site_name': f"Cluster_{iteration}"
                    }])
                ], ignore_index=True)

                for src in members:
                    distance = haversine(
                        (df_source[df_source[source_id] == src]['lat'].values[0],
                         df_source[df_source[source_id] == src]['lon'].values[0]),
                        (centroid_lat, centroid_lon)
                    )
                    df_cost_matrix.loc[df_cost_matrix[sink_id] == centroid_id, src] = distance

                    for _, sink_row in df_sink[df_sink[sink_id] != centroid_id].iterrows():
                        original_cost = df_cost_matrix.loc[df_cost_matrix[sink_id] == sink_row[sink_id], src].values[0]
                        df_cost_matrix.loc[df_cost_matrix[sink_id] == sink_row[sink_id], centroid_id] = original_cost / len(members)

            # Post-process results
            for _, row in current_results.iterrows():
                if row['sink_id'].startswith('centroid_'):
                    centroid = row['sink_id']
                    downstream = current_results[current_results['source_id'] == centroid]
                    for _, ds_row in downstream.iterrows():
                        all_results.append({
                            'source_id': row['source_id'],
                            'sink_id': ds_row['sink_id'],
                            'co2_transported': row['co2_transported'],
                            'path': f"{row['source_id']}→{centroid}→{ds_row['sink_id']}"
                        })
                else:
                    all_results.append(row.to_dict())

        if not all_results:
            return pd.DataFrame(columns=['source_id', 'sink_id', 'co2_transported', 'path'])
        return pd.DataFrame(all_results)[['source_id', 'sink_id', 'co2_transported', 'path']]

    except Exception as e:
        print(f"Iterative optimization failed: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame(columns=['source_id', 'sink_id', 'co2_transported', 'path'])


In [192]:
import pandas as pd
import numpy as np

# Example Data Preparation
def prepare_example_data():
    # Example Sources (Cement Plants)
    source_data = {
        'id': ['1897479', '1897478', '32438922', '1897504', '32438914'],
        'name': ['CEMEX Mexico SA de CV Puebla 1', 
                'CEMEX Mexico SA de CV Hidalgo 1',
                'Holcim Mexico SA de CV Veracruz 1',
                'Holcim Mexico SA de CV Tabasco 1',
                'Cementos Fortaleza Hidalgo 2'],
        'emission': [2.794440e+06, 1.036068e+06, 9.600968e+05, 8.838254e+05, 8.025427e+05],
        'lat': [18.967968, 19.996589, 18.857600, 17.651499, 20.011934],
        'lon': [-97.961858, -99.210844, -97.041208, -92.443296, -99.178466]
    }
    df_source = pd.DataFrame(source_data)

    # Example Sinks (Storage Sites)
    sink_data = {
        'id': ['10346', '10347', '10348', '10349', '10350'],
        'sum_mid': [4.300000e+08, 1.040000e+09, 1.920000e+09, 1.970000e+09, 5.400000e+08],
        'latitude': [25.642778, 25.664722, 25.758611, 25.779444, 25.809444],
        'longitude': [-99.071389, -98.954444, -98.580000, -98.438611, -97.863611],
        'site_name': ['Burgos B1A', 'Burgos B1B', 'Burgos B1C', 'Burgos B1D', 'Burgos B1E']
    }
    df_sink = pd.DataFrame(sink_data)

    # Example Cost Matrix (Distances between sources and sinks)
    cost_matrix_data = {
        'Unnamed: 0': ['10346', '10347', '10348', '10349', '10350'],
        '1897479': [345.817296, 344.745828, 348.487164, 340.211268, 336.485808],
        '1897478': [332.455428, 329.412024, 333.153360, 324.877428, 321.152004],
        '32438922': [373.947948, 372.876480, 376.617816, 368.341884, 364.616460],
        '1897504': [521.441244, 520.369740, 524.111148, 515.835180, 512.109720],
        '32438914': [331.633836, 329.152284, 332.893620, 324.617688, 320.892264]
    }
    df_cost_matrix = pd.DataFrame(cost_matrix_data)

    return df_source, df_sink, df_cost_matrix



In [193]:
# Prepare data (with string IDs)
df_source, df_sink, df_cost_matrix = prepare_example_data()

# Run optimization
results = iterative_clustered_network_optimization(
    df_source=df_source,
    df_sink=df_sink,
    df_cost_matrix=df_cost_matrix,
    source_id='id',
    sink_id='id',
    source_capacity='emission',
    sink_capacity='sum_mid',
    emission_cost=50,
    transport_method='pipe',
    quantity_cost_segments=quantity_cost_segments,
    capture_cost=30,
    stock_cost=10,
    max_iter=5
)

print(results[['source_id', 'sink_id', 'co2_transported', 'path']])

Empty DataFrame
Columns: [source_id, sink_id, co2_transported, path]
Index: []
